In [1]:
# taken from ngboost
# https://github.com/stanfordmlgroup/ngboost/blob/master/figures/toy.py
import numpy as np


np.random.seed(1)
def gen_data(n=50, bound=1, deg=3, beta=1, noise=0.9, intcpt=-1):
    x = np.linspace(-bound, bound, n)[:, np.newaxis]
    h = np.linspace(-bound, bound, n)[:, np.newaxis]
    e = np.random.randn(*x.shape) * (0.1 + 10 * np.abs(x))
    y = 50 * (x**deg) + h * beta + noise * e + intcpt
    return x, y.squeeze(), np.c_[h, np.ones_like(h)]

# training
x_tr, y_tr, _ = gen_data(n=100)
# Prepend a bias column to x_tr
# adapted from ngboost for clarity
#from sklearn.preprocessing import PolynomialFeatures
#poly_transform = PolynomialFeatures(1)
#x_tr = poly_transform.fit_transform(x_tr)
x_tr = np.hstack([np.ones((x_tr.shape[0], 1)), x_tr])


x_te, y_te, _ = gen_data(n=1000, bound=1.3)
x_te = np.hstack([np.ones((x_te.shape[0], 1)), x_te])

In [ ]:
from ngboost.distns import Normal
from ngboost.evaluation import *
from ngboost.learners import default_tree_learner
from ngboost.ngboost import NGBoost
from ngboost.scores import MLE

BLK = 2

# Default arguments
n_estimators = 1 + BLK * 100
lr = 0.03
minibatch_frac = 0.1
natural = True

ngb = NGBoost(
    Base=default_tree_learner,
    Dist=Normal,
    Score=MLE,
    n_estimators=n_estimators,
    learning_rate=lr,
    natural_gradient=natural,
    minibatch_frac=minibatch_frac,
    verbose=True,
)

blk = int(n_estimators / 100)
ngb.fit(x_tr, y_tr)

In [ ]:
import matplotlib.pyplot as plt


preds = ngb.pred_dist(x_te)


filenames = []
all_preds = ngb.staged_pred_dist(x_te)
for i, preds in enumerate(all_preds):
    if i % blk != 0:
        continue
    plt.figure(figsize=(6, 3))
    plt.scatter(x_tr[:, 1], y_tr, color="black", marker=".", alpha=0.5)
    plt.plot(x_te[:, 1], preds.loc, color="black", linestyle="-", linewidth=1)
    plt.plot(
        x_te[:, 1],
        preds.loc - 1.96 * preds.scale,
        color="black",
        linestyle="--",
        linewidth=0.3,
    )
    plt.plot(
        x_te[:, 1],
        preds.loc + 1.96 * preds.scale,
        color="black",
        linestyle="--",
        linewidth=0.3,
    )
    plt.ylim([-75, 75])
    plt.axis("off")
    plt.title(
        "%s Gradient: %02d%% fit"
        % ("Natural" if natural else "Ordinary", i / blk)
    )
    plt.tight_layout()

    filenames.append("./figures/anim/toy%d.png" % i)
    print("Saving ./figures/anim/toy%d" % i)
    plt.savefig("./figures/anim/toy%d.png" % i)
    if i % 100 == 0:
        plt.savefig("./figures/anim/toy%d.pdf" % i)



In [4]:
# create a gif
import imageio

images = []
for filename in filenames:
    images.append(imageio.v2.imread(filename))
for _ in range(10):
    images.append(imageio.v2.imread(filename))
imageio.mimsave(f"./figures/toy_{'natural' if natural else 'ordinary'}.gif", images, fps=5)